In [10]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model, Model

In [11]:
# Load your custom model
model_path = 'brats_3d_total.hdf5'
custom_model = load_model(model_path, compile=False)

In [12]:
# Summary of the loaded model to understand the architecture
custom_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 3)]                                                           
                                                                                                  
 conv3d_19 (Conv3D)             (None, 128, 128, 12  1312        ['input_2[0][0]']                
                                8, 16)                                                            
                                                                                                  
 dropout_9 (Dropout)            (None, 128, 128, 12  0           ['conv3d_19[0][0]']              
                                8, 16)                                                      

                                                                                                  
 conv3d_31 (Conv3D)             (None, 32, 32, 32,   221248      ['concatenate_5[0][0]']          
                                64)                                                               
                                                                                                  
 dropout_15 (Dropout)           (None, 32, 32, 32,   0           ['conv3d_31[0][0]']              
                                64)                                                               
                                                                                                  
 conv3d_32 (Conv3D)             (None, 32, 32, 32,   110656      ['dropout_15[0][0]']             
                                64)                                                               
                                                                                                  
 conv3d_tr

In [13]:
# Freeze layers if needed (optional)
for layer in custom_model.layers:
    layer.trainable = False

In [14]:
# Specify the layer from which you want to extract features
feature_extractor = Model(inputs=custom_model.input, outputs=custom_model.layers[-2].output)

In [15]:
import os
def sorted_dir(directory):
    items=os.listdir(directory)
    sorted_items = sorted(items)
    return sorted_items

In [16]:
# Paths to the image and mask directories
image_dir = 'Brats2020_input_train_data_3channels/images/'
mask_dir = 'Brats2020_input_train_data_3channels/masks/'

print(image_dir)

Brats2020_input_train_data_3channels/images/


In [17]:
train_img_list = sorted_dir(image_dir)
train_mask_list = sorted_dir(mask_dir)

In [19]:
def load_img(img_list, img_dir):
    images = []
    image_names = []
    for i, image_name in enumerate(img_list):    
        if image_name.endswith('.npy'):
            image = np.load(os.path.join(img_dir, image_name))
            images.append(image)
            image_names.append(image_name)
    images = np.array(images)
    return images, image_names

In [26]:
# List of images and masks
# image_files = sorted([image_dir + f'image_{i}.npy' for i in range(218)])
# mask_files = sorted([mask_dir + f'mask_{i}.npy' for i in range(218)])

In [27]:
# Function to load images and masks from .npy files
# def load_data(image_files, image_dir):
#     images = load_img(image_files, image_dir)
# #     masks = [np.load(file) for file in mask_files]
#     # Reshape and normalize images if needed
# #     images = np.array(images).astype('float32') / 255.0  # Normalize images to [0, 1]
# #     masks = np.array(masks)
#     # Expand dimensions if grayscale to add channel dimension
# #     images = np.expand_dims(images, axis=-1)  # Shape becomes (num_samples, 128, 128, 128, 1)
#     return images

In [20]:
# Load images and masks
images, image_names = load_img(train_img_list, image_dir)

In [30]:
len(images)

218

In [21]:
# Load images and masks
masks, mask_names = load_img(train_mask_list, mask_dir)

In [22]:
print(f"Loaded {len(images)} images and {len(masks)} masks.")

Loaded 218 images and 218 masks.


In [33]:
# Extract features using the custom model
features = feature_extractor.predict(images, batch_size=2)

# Flatten the features if they are multidimensional (e.g., (num_samples, feature_dim))
features_flattened = features.reshape(features.shape[0], -1)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
# Convert features to DataFrame
features_df = pd.DataFrame(features_flattened)

In [ ]:
features_df['Image_Name'] = image_names

In [ ]:
columns = ['Image_Name'] + [f'Feature_{i+1}' for i in range(features_flattened.shape[1])]
features_df.columns = columns

In [ ]:
# Save features to CSV file
features_df.to_csv('dl_extracted_features_with_names.csv', index=False)

print("Features successfully extracted and saved to 'dl_extracted_features_with_names.csv'.")